## Als erstes die Framewerks importieren, die Daten laden und als Pandas DataFrames verwenden

In [2]:
import pandas as pd # für Datenimport
import numpy as np
import os # für Dateipfade
import sys
from datetime import datetime # um Datum und Uhrzeit im Überblick auszugeben



# Dateinamen definieren und Dateipfade erstellen 
customers_file = "data/customers.csv"
customers_path = os.path.join(os.getcwd(), customers_file)
offers_file = "data/offers.csv"
offers_path = os.path.join(os.getcwd(), offers_file)
contacts_file = "data/contacts.csv"
contacts_path = os.path.join(os.getcwd(), contacts_file)

# Datensätze als Pandas-Dataframes laden 
customers_data = pd.read_csv(customers_path)
offers_data = pd.read_csv(offers_path)
contacts_data = pd.read_csv(contacts_path)

## DataFrames aufräumen

### Datensatz Contacts

In [3]:
# Die ersten beiden Spalten von contacts_data streichen
# musste erste Spalte der CSV anpassen um greifen zu können
contacts_data = contacts_data.drop(['Unnamed', 'Unnamed: 0'], axis=1) 

# contacts_data.index = contacts_data["person"] # person zum Index machen

### Datensatz Customers

In [4]:
# For-Schleife zum Vergleichen der Werte in den ersten beiden Spalten
# Können die ersten beiden Spalten gestrichen werden?
for index, row in customers_data.iterrows():
    if row['Unnamed: 0'] - row['Unnamed: 0.1'] != 0:
        print(f"Fehler in Zeile {index+1}: Wert von Spalte B ist ungleich Spalte A")

# Entscheidung dafür, die ersten beiden Spalten von customers_data zu streichen
# drop-Funktion gibt es ein neues DataFrame aus
customers_data = customers_data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1) 

# customers_data.index = customers_data["cust_id"] # ID zum Index machen

# Datumsangaben in customers.csv zu einem datetime64 Object aus Pandas umwandeln, 
# um danach weitere Berechnungen machen zu können
customers_data["became_member_on"] = pd.to_datetime(customers_data["became_member_on"], format='%Y%m%d')

In [5]:
# Laut Frau Anderl ist in der Spalte "age" 118 lediglich ein Platzhalter für einen leeren Wert
# Um Durchschnitt, Median und co. nicht zu verfälschen: Umwandeln zu NaN
customers_data.loc[customers_data['age'] == 118, 'age'] = np.nan

In [6]:
customers_data[(customers_data["age"] == 118)]

,gender,age,cust_id,became_member_on,income


### Datensatz Offers

In [7]:
offers_data = offers_data.drop(["Unnamed"], axis=1)

## Überblick über die Datensätze

### Contacts

In [8]:
contacts_data.info()
contacts_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280468 entries, 0 to 280467
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  280468 non-null  object
 1   event   280468 non-null  object
 2   time    280468 non-null  int64 
 3   type    280468 non-null  object
 4   val     280468 non-null  object
dtypes: int64(1), object(4)
memory usage: 10.7+ MB


,person,event,time,type,val
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,offer_id,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,offer_id,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,offer_id,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,offer_id,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,offer_id,4d5c57ea9a6940dd891ad53e9dbe8da0


In [9]:
contacts_data['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [10]:
contacts_data[~contacts_data.event.str.contains("transaction")]["event"].unique()

array(['offer received', 'offer viewed', 'offer completed'], dtype=object)

In [11]:
# Gib mir alle Zeilen aus, die NICHT "transaction" enthalten und zähle dann, wie oft welcher
# Wert in der Spalte "val" vorkommt
contacts_data[~contacts_data.event.str.contains("transaction")]["val"].value_counts()

fafdcd668e3743c1bb461111dcafc2a4    20241
2298d6c36e964ae4a3e7e9706d1fb8c2    20139
f19421c1d4aa40978ebb69ca19b0e20d    19131
4d5c57ea9a6940dd891ad53e9dbe8da0    18222
ae264e3637204a6fb9bb56bc8210ddfd    18062
9b98b8c7a33c4b65b9aebfe6a799e6d9    16202
2906b810c7d4411798c6938adc9daaa5    15767
0b1e1539f2cc45b7b9fa7c272da2e1d7    13751
Name: val, dtype: int64

In [12]:
received_data = contacts_data[contacts_data.event.str.contains("offer received")]["val"].value_counts()
offers_data["received"] = offers_data["id"].map(received_data)

In [13]:
viewed_data = contacts_data[contacts_data.event.str.contains("offer viewed")]["val"].value_counts()
offers_data["viewed"] = offers_data["id"].map(viewed_data)

In [14]:
completed_data = contacts_data[contacts_data.event.str.contains("offer completed")]["val"].value_counts()
offers_data["completed"] = offers_data["id"].map(completed_data)

In [15]:
offers_data

,reward,channels,difficulty,duration,offer_type,id,received,viewed,completed
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,7658,6716,3688
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,7593,7298,3331
2,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,7677,4171,4354
3,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,7668,2663,3420
4,3,"['web', 'email', 'mobile', 'social']",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,7646,7337,5156
5,2,"['web', 'email', 'mobile', 'social']",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,7597,7327,5317
6,5,"['web', 'email', 'mobile', 'social']",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,7571,7264,4296
7,2,"['web', 'email', 'mobile']",10,7,discount,2906b810c7d4411798c6938adc9daaa5,7632,4118,4017


### Customers

In [16]:
customers_data.info()
customers_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   gender            14825 non-null  object        
 1   age               14825 non-null  float64       
 2   cust_id           17000 non-null  object        
 3   became_member_on  17000 non-null  datetime64[ns]
 4   income            14825 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 664.2+ KB


,gender,age,cust_id,became_member_on,income
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


### Offers

In [17]:
offers_data.info()
offers_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      8 non-null      int64 
 1   channels    8 non-null      object
 2   difficulty  8 non-null      int64 
 3   duration    8 non-null      int64 
 4   offer_type  8 non-null      object
 5   id          8 non-null      object
 6   received    8 non-null      int64 
 7   viewed      8 non-null      int64 
 8   completed   8 non-null      int64 
dtypes: int64(6), object(3)
memory usage: 704.0+ bytes


,reward,channels,difficulty,duration,offer_type,id,received,viewed,completed
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,7658,6716,3688
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,7593,7298,3331
2,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,7677,4171,4354
3,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,7668,2663,3420
4,3,"['web', 'email', 'mobile', 'social']",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,7646,7337,5156


## Auswertung

Hier schauen wir uns wir uns an, wer wie oft kontaktiert wurde. 
- Wie ist der Durchschnitt?
- Wer sind die 10 Personen, die am häufigsten konktakiert wurden?

Weitergehend möglich: 
- Welche Aussage können wir über die top 10 Personen treffen?
- Welche Personen wurden noch nicht kontaktiert?

In [18]:
contacts_per_person = contacts_data['person'].value_counts()
contacts_per_person

94de646f7b6041228ca7dec82adb97d2    51
8dbfa485249f409aa223a2130f40634a    49
d0a80415b84c4df4908b8403b19765e3    48
b1f4ece7d49342628a9ed77aee2cde58    46
5e60c6aa3b834e44b822ea43a3efea26    46
                                    ..
b8f309e5c6b34d1b8158468dcf6cbdec     1
3389da76083a40d399af9642163b5cd3     1
1b978feee51d4afe83ecde1ae78907cc     1
724384e51b674088ad22a0ed69405c02     1
da7a7c0dcfcb41a8acc7864a53cf60fb     1
Name: person, Length: 16996, dtype: int64

In [19]:
customers_data["number of contacts"] = customers_data["cust_id"].map(contacts_per_person)
customers_data.head()

,gender,age,cust_id,became_member_on,income,number of contacts
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,21.0
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,5.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,8.0
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,16.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,8.0


In [20]:
customers_data["number of contacts"].describe()

count    16996.00000
mean        16.50200
std          6.99651
min          1.00000
25%         11.00000
50%         16.00000
75%         21.00000
max         51.00000
Name: number of contacts, dtype: float64

In [21]:
customers_data[customers_data["number of contacts"].isnull()]

,gender,age,cust_id,became_member_on,income,number of contacts
11367,F,53.0,96fa841c065d482bad03b19a7acfe95d,2017-10-21,94000.0,NaN
12124,M,53.0,f526f655ab8e48f2873dcda54a301afd,2017-09-19,57000.0,NaN
13853,M,39.0,fafcd6ee168140fbbb5da43be1d3daa7,2018-05-23,41000.0,NaN
14607,NaN,NaN,7ecfc592171f4844bdc05bdbb48d3847,2018-03-01,NaN,NaN


Wir wollten uns jetzt anschauen, wie lange es dauert, bis Angebote eingelöst werden.

In [29]:

contacts_data.loc[contacts_data["event"] == "offer received"]["time"].unique()

array([  0, 168, 336, 408, 504, 576])

In [ ]:
contacts_data.loc[contacts_data["event"] == "offer received"]

## Visualisierungen